# CRAWLER AND SCRAPER OF ACM DIGITAL LIBRARY

## NEEDED LIBRARIES

In [1]:
import pandas as pd
import cfscrape
from lxml import etree

## SET OF KEYWORDS

In [2]:
keywords = ['Cloud Security']

## INSTANTIATING THE CRAWLER AND DEFINING AN REALISTIC HEADER 

In [3]:
scraper = cfscrape.create_scraper()

header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15'}

## READING OLD DATASET FOR APPENDING or CREATE A NEW (EMPTY) DATAFRAME

In [7]:
# df= pd.read_csv('20191001_acm_dl_ddos_papers.csv').drop(['Unnamed: 0'],axis=1)
df = pd.DataFrame(columns=['links'])

## LOOP FOR CRAWLING AND SCRAPPING (IN 2 LEVELS) ALL ENTRIES RELATED TO THE SET OF KEYWORDS 

In [8]:
for keyword in keywords:
    page = 1 
    while True:
        print(page,keyword)
        
        keyword = keyword.replace(' ','+')
        url = "https://dl.acm.org/results.cfm?query=%22"+keyword+"%22&start="+str((page-1)*20)+"&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore"
        print(url)
        
        scraped_html=scraper.get(url,headers=header).content
        html = etree.HTML(scraped_html)
        
        max_entries = html.xpath("//div[@class='pagerange']/text()")[0].split()[-1]
        if (page-1)*20 > int(max_entries):
            break
        
        df_temp = pd.DataFrame()
        
        #Crawler first level
        title = []
        links = []
        publication_date = []
        venue=[]

        for entry in html.xpath("//div[@class='title']/a"):
            try:
                title.append(entry.xpath("text()")[0])
            except:
                title.append("")

            try: 
                links.append('https://dl.acm.org/'+entry.xpath("@href")[0]+'&preflayout=flat')
            except:
                links.append("")

        for entry in html.xpath("//div[@class='source']"): 
            try:
                publication_date.append(entry.xpath("span[@class='publicationDate']/text()")[0])
            except:
                publication_date.append("")

            try:
                venue.append(entry.xpath("span[@style='padding-left:10px']/text()")[0])
            except:
                venue.append("")

        df_temp = pd.DataFrame({'title': title,'links': links,'publication_date': publication_date,'venue': venue,})
    
        #Crawler second level
        for link in df_temp['links']:
            print(link)
            # CHECK IF THE LINK ALREADY EXIST
            if len(df[df['links']==link]) >0:
                df_temp.drop(df_temp.loc[df_temp['links']==link].index, inplace=True)
                print("duplicated")
                continue
                
            
            scraped_html=scraper.get(link,headers=header).content
            html = etree.HTML(scraped_html)

            index_current = df_temp.index[df_temp['links'] == link][0]

            try:
                df_temp.at[index_current, 'link4download'] = "https://dl.acm.org/" + html.xpath("//table[@class='medium-text']/tr/td/a/@href")[0]
            except:
                df_temp.at[index_current, 'link4download'] = ""

            try:
                df_temp.at[index_current, 'num_citations'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Citation' in x][0].replace('\n','').replace('·\xa0','').replace('Citation Count: ','')
            except:
                df_temp.at[index_current, 'num_citations'] =""

            try:
                df_temp.at[index_current, 'num_downloads_cu'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Downloads (cumulative)' in x][0].replace('\n','').replace('·\xa0','').replace('Downloads (cumulative): ','')
            except:
                df_temp.at[index_current, 'num_downloads_cu'] =""

            try:
                df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/p/text()")[0]
            except:
                try: 
                    df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/text()")[0]
                except:
                    try:
                        df_temp.at[index_current, 'abstract'] = html.xpath("//div[@class='flatbody']/div/text()")[0].replace('\n','')
                    except:
                        df_temp.at[index_current, 'abstract'] = ""
        
        df = df.append(df_temp,ignore_index=True)
        display(df.tail(1))
        
        page = page + 1

1 Cloud Security
https://dl.acm.org/results.cfm?query=%22Cloud+Security%22&start=0&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore
https://dl.acm.org/citation.cfm?id=2046668&preflayout=flat
https://dl.acm.org/citation.cfm?id=2046662&preflayout=flat
https://dl.acm.org/citation.cfm?id=2046674&preflayout=flat
https://dl.acm.org/citation.cfm?id=2331767&preflayout=flat
https://dl.acm.org/citation.cfm?id=2536164&preflayout=flat
https://dl.acm.org/citation.cfm?id=2459981&preflayout=flat
https://dl.acm.org/citation.cfm?id=2668278&preflayout=flat
https://dl.acm.org/citation.cfm?id=3231831&preflayout=flat
https://dl.acm.org/citation.cfm?id=2527013&preflayout=flat
https://dl.acm.org/citation.cfm?id=2752500&preflayout=flat
https://dl.acm.org/citation.cfm?id=2181197&preflayout=flat
https://dl.acm.org/citation.cfm?id=3363371&preflayout=flat
https://dl.acm.org/citation.cfm?id=3285012&preflayout=flat
https://dl.acm.org/citation.cfm?id=2818672&preflayout=flat
https://dl.acm.org/citation

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,abstract,link4download,links,num_citations,num_downloads_cu,publication_date,title,venue
19,Though there has been much study of informatio...,https://dl.acm.org/ft_gateway.cfm?id=3267843&f...,https://dl.acm.org/citation.cfm?id=3267843&pre...,0,115,October 2018,I Heard It through the Firewall: Exploiting Cl...,SoCC '18: Proceedings of the ACM Symposium on ...


2 Cloud+Security
https://dl.acm.org/results.cfm?query=%22Cloud+Security%22&start=20&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore
https://dl.acm.org/citation.cfm?id=2955208&preflayout=flat
https://dl.acm.org/citation.cfm?id=3148098&preflayout=flat
https://dl.acm.org/citation.cfm?id=2752972&preflayout=flat
https://dl.acm.org/citation.cfm?id=2883008&preflayout=flat
https://dl.acm.org/citation.cfm?id=3103172&preflayout=flat
https://dl.acm.org/citation.cfm?id=3022319&preflayout=flat
https://dl.acm.org/citation.cfm?id=3349487&preflayout=flat
https://dl.acm.org/citation.cfm?id=3360287&preflayout=flat
https://dl.acm.org/citation.cfm?id=3007875&preflayout=flat
https://dl.acm.org/citation.cfm?id=2381915&preflayout=flat
https://dl.acm.org/citation.cfm?id=2371564&preflayout=flat
https://dl.acm.org/citation.cfm?id=3232181&preflayout=flat
https://dl.acm.org/citation.cfm?id=3176317&preflayout=flat
https://dl.acm.org/citation.cfm?id=2414504&preflayout=flat
https://dl.acm.org/citatio

,abstract,link4download,links,num_citations,num_downloads_cu,publication_date,title,venue
39,Cloud storage services allow a simple collabor...,https://dl.acm.org/ft_gateway.cfm?id=2998446&f...,https://dl.acm.org/citation.cfm?id=2998446&pre...,0,67,October 2016,A Secure Collaborative Network Protocol,LANC '16: Proceedings of the 9th Latin America...


...

9 Cloud+Security
https://dl.acm.org/results.cfm?query=%22Cloud+Security%22&start=160&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore
https://dl.acm.org/citation.cfm?id=3116211&preflayout=flat
https://dl.acm.org/citation.cfm?id=2381913&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=2381939&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=3098026&preflayout=flat
https://dl.acm.org/citation.cfm?id=2768566&preflayout=flat
https://dl.acm.org/citation.cfm?id=3232140&preflayout=flat
https://dl.acm.org/citation.cfm?id=2491783&preflayout=flat
https://dl.acm.org/citation.cfm?id=3297296&preflayout=flat
https://dl.acm.org/citation.cfm?id=2905253&preflayout=flat
https://dl.acm.org/citation.cfm?id=3219862&preflayout=flat
https://dl.acm.org/citation.cfm?id=2898445&preflayout=flat
https://dl.acm.org/citation.cfm?id=3267362&preflayout=flat
https://dl.acm.org/citation.cfm?id=2613088&preflayout=flat
https://dl.acm.org/citation.cfm?id=2676688&preflayout=flat
htt

,abstract,link4download,links,num_citations,num_downloads_cu,publication_date,title,venue
160,It is our great pleasure to welcome you to the...,https://dl.acm.org/author_page.cfm?id=81100222...,https://dl.acm.org/citation.cfm?id=1871929&pre...,26,"6,088",October 2010,CloudDB '10: Proceedings of the second interna...,


10 Cloud+Security
https://dl.acm.org/results.cfm?query=%22Cloud+Security%22&start=180&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore
https://dl.acm.org/citation.cfm?id=2991109&preflayout=flat
https://dl.acm.org/citation.cfm?id=2523138&preflayout=flat
https://dl.acm.org/citation.cfm?id=2295138&preflayout=flat
https://dl.acm.org/citation.cfm?id=3134636&preflayout=flat
https://dl.acm.org/citation.cfm?id=2517488&preflayout=flat
https://dl.acm.org/citation.cfm?id=2857552&preflayout=flat
https://dl.acm.org/citation.cfm?id=2732516&preflayout=flat
https://dl.acm.org/citation.cfm?id=2664168&preflayout=flat
https://dl.acm.org/citation.cfm?id=3306643&preflayout=flat
https://dl.acm.org/citation.cfm?id=2408793&preflayout=flat
https://dl.acm.org/citation.cfm?id=3275470&preflayout=flat


,abstract,link4download,links,num_citations,num_downloads_cu,publication_date,title,venue
171,Application and platform security has always b...,https://dl.acm.org/ft_gateway.cfm?id=3275470&f...,https://dl.acm.org/citation.cfm?id=3275470&pre...,0,93,October 2018,Drishti: Disaggregated and Interoperable Secur...,SoCC '18: Proceedings of the ACM Symposium on ...


11 Cloud+Security
https://dl.acm.org/results.cfm?query=%22Cloud+Security%22&start=200&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore


In [9]:
len(df)

172

In [13]:
df['publication_date'].value_counts()

April 2017        6
August 2018       6
October 2018      6
June 2015         5
May 2017          5
October 2011      5
March 2018        5
May 2013          5
November 2014     5
November 2019     5
October 2015      5
April 2015        4
August 2017       4
November 2013     4
November 2017     3
December 2015     3
May 2018          3
March 2017        3
March 2012        3
September 2018    3
October 2012      3
December 2016     3
November 2016     3
January 2018      3
May 2016          2
October 2016      2
November 2009     2
December 2017     2
November 2011     2
August 2012       2
                 ..
April 2016        2
December 2018     2
April 2013        1
August 2013       1
December 2013     1
January 2013      1
May 2015          1
April 2018        1
October 2014      1
February 2013     1
June 2014         1
July 2019         1
April 2012        1
June 2013         1
October 2013      1
March 2014        1
December 2012     1
March 2013        1
May 2019          1


## SAVING FINDINGS TO PROCESS OFFLINE

In [11]:
df.to_csv('20191111_acm_dl_cloud_security.csv')

In [12]:
df.head()

,abstract,link4download,links,num_citations,num_downloads_cu,publication_date,title,venue
0,Most of the problems facing the designers of a...,https://dl.acm.org/ft_gateway.cfm?id=2046668&f...,https://dl.acm.org/citation.cfm?id=2046668&pre...,0,"2,103",October 2011,What's different about security in a public cl...,CCSW '11: Proceedings of the 3rd ACM workshop ...
1,Cloud computing has become increasingly popula...,https://dl.acm.org/ft_gateway.cfm?id=2046662&f...,https://dl.acm.org/citation.cfm?id=2046662&pre...,0,845,October 2011,Clouds and their discontents,CCSW '11: Proceedings of the 3rd ACM workshop ...
2,Can the cloud truly be secured? Can enterprise...,https://dl.acm.org/ft_gateway.cfm?id=2046674&f...,https://dl.acm.org/citation.cfm?id=2046674&pre...,0,"1,975",October 2011,Cloud security: myth or reality?,CCSW '11: Proceedings of the 3rd ACM workshop ...
3,Cloud computing is a long fantasized visualiza...,https://dl.acm.org/ft_gateway.cfm?id=2331767&f...,https://dl.acm.org/citation.cfm?id=2331767&pre...,6,"2,259",March 2012,Different facets of security in the cloud,CNS '12: Proceedings of the 15th Communication...
4,It is recognized that the most important obsta...,https://dl.acm.org/ft_gateway.cfm?id=2536164&f...,https://dl.acm.org/citation.cfm?id=2536164&pre...,1,341,October 2013,A general approach for a trusted deployment of...,MEDES '13: Proceedings of the Fifth Internatio...


## DEBUGGING ERROS 1ST LEVEL CRAWLER

In [ ]:
url="https://dl.acm.org/results.cfm?query=%22denial+of+service%22&start=300&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore"

In [ ]:
scraped_html=scraper.get(url,headers=header).content
print(scraped_html)

In [ ]:
html = etree.HTML(scraped_html)

df_temp = pd.DataFrame()
title = []
links = []
publication_date = []
venue=[]

for entry in html.xpath("//div[@class='title']/a"):
    try:
        title.append(entry.xpath("text()")[0])
    except:
        title.append("")
    
    try: 
        links.append('https://dl.acm.org/'+entry.xpath("@href")[0]+'&preflayout=flat')
    except:
        links.append("")

for entry in html.xpath("//div[@class='source']"): 
    try:
        publication_date.append(entry.xpath("span[@class='publicationDate']/text()")[0])
    except:
        publication_date.append("")
    
    try:
        venue.append(entry.xpath("span[@style='padding-left:10px']/text()")[0])
    except:
        venue.append("")

df_temp = pd.DataFrame({'title': title,'links': links,'publication_date': publication_date,'venue': venue,})

## DEBUGING ERROS 2nd LEVEL CRAWLER

In [ ]:
link = "https://dl.acm.org/citation.cfm?id=372148&preflayout=flat"

In [ ]:
scraped_html=scraper.get(link,headers=header).content
print(scraped_html)

In [ ]:
html = etree.HTML(scraped_html)

df_temp = pd.DataFrame()
index_current = 0

try:
    df_temp.at[index_current, 'link4download'] = "https://dl.acm.org/" + html.xpath("//table[@class='medium-text']/tr/td/a/@href")[0]
except:
    df_temp.at[index_current, 'link4download'] = ""

try:
    df_temp.at[index_current, 'num_citations'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Citation' in x][0].replace('\n','').replace('·\xa0','').replace('Citation Count: ','')
except:
    df_temp.at[index_current, 'num_citations'] =""

try:
    df_temp.at[index_current, 'num_downloads_cu'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Downloads (cumulative)' in x][0].replace('\n','').replace('·\xa0','').replace('Downloads (cumulative): ','')
except:
    df_temp.at[index_current, 'num_downloads_cu'] =""

try:
    df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/p/text()")[0]
except:
    try: 
        df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/text()")[0]
    except:
        try:
            df_temp.at[index_current, 'abstract'] = html.xpath("//div[@class='flatbody']/div/text()")[0].replace('\n','')
        except:
            df_temp.at[index_current, 'abstract'] = ""

df_temp

<h2 align='center'>###############################################################<br>###############################################################<br>OFFLINE PROCESSING!!!</h2>

## READING THE DATAFRAME 

In [ ]:
df= pd.read_csv('20191001_acm_dl_ddos_papers.csv').drop(['Unnamed: 0'],axis=1)

## QUICK PREVIEW OF THE DATA

In [ ]:
df.head()

## ISOLATING THE ID IN A COLUMN

In [ ]:
df['id'] = df['links'].apply(lambda x: x.replace("https://dl.acm.org/citation.cfm?id=","").replace("&preflayout=flat",""))

## CREATING SOME EMPTY COLUMNS FOR MANUAL CLASSIFICATION

In [ ]:
df['dm']=""
df['attack_type']= ""
df['defence_deployment']=""
df['not_attack_type']=""
df['attack_impact']=""
df['attack_infra']=""
df['overview']= ""

## EXPORTING COLUMNS TO CSV FOR MANUAL CLASSIFICATION

In [ ]:
df[['id','abstract','overview','dm','attack_type','defence_deployment','attack_infra','attack_impact','not_attack_type']].to_excel('abstracts4classifying.xlsx')